In [4]:
from supportfunctions import finiteDiff
import SolveLinSys
from scipy.interpolate import RegularGridInterpolator
from scipy import fft, arange, signal
from scipy import interpolate
from scipy.optimize import curve_fit
import scipy.optimize as optim
import scipy.io as sio
import matplotlib.mlab
from matplotlib.colors import SymLogNorm
from matplotlib import rcParams
from matplotlib import pyplot as plt
from scipy.fft import fft, fftfreq
from scipy.integrate import solve_ivp
import pickle
import pandas as pd
import numpy as np
import sys
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

sys.path.append("../src/")
rcParams["figure.figsize"] = (8, 5)
rcParams["savefig.bbox"] = 'tight'
rcParams["font.size"] = 18


def PDESolver(stateSpace, A, B1, B2, B3, C1, C2, C3, D, v0,
              ε=1, tol=-10):

    A = A.reshape(-1, 1, order='F')
    B = np.hstack([B1.reshape(-1, 1, order='F'), B2.reshape(-1,
                  1, order='F'), B3.reshape(-1, 1, order='F')])
    C = np.hstack([C1.reshape(-1, 1, order='F'), C2.reshape(-1,
                  1, order='F'), C3.reshape(-1, 1, order='F')])
    D = D.reshape(-1, 1, order='F')
    v0 = v0.reshape(-1, 1, order='F')
    out = SolveLinSys.solveFT(stateSpace, A, B, C, D, v0, ε, tol)

    return out


In [5]:
from os.path import isfile, join
from os import listdir

mypath="./data/guess1/"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

@interact
def present(Fnum=(0, 100), Cnum=(0, 20), cearth=[35.0], tauc=[6603.0], fraction=np.array((0.05, 0.1)), epsilon=np.array((0.05, 0.1)), maxiter=np.array((5000, 30000, 60000)), Xminarr=[[0.0, 200.0, 0.0]], Xmaxarr=[[10.0, 400.0, 2000.0], [15.0, 400.0, 2000.0], [17.5, 400.0, 2000.0], [20.0, 400.0, 2000.0]], hXarr=[[0.2, 4.0, 40.0],[0.1, 2.0, 20.0]]):
    dataname = f"./data/HJB_Bin.py/data_{cearth}_{tauc}_{maxiter}_{fraction}_{epsilon}_{Xminarr}_{Xmaxarr}_{hXarr}"
    # print(dataname)
    res = pickle.load(open(dataname, "rb"))

    v0 = res["v0"]
    T_grid = res["T"]
    C_grid = res["C"]
    F_grid = res["F"]
    Ca = res["Ca"]

    hT = T_grid[1]-T_grid[0]
    hC = C_grid[1]-C_grid[0]
    hF = F_grid[1]-F_grid[0]

    dvdC = finiteDiff(v0, 1, 1, hC)
    dvdF = finiteDiff(v0, 2, 1, hF)

    plt.subplots(1, 3, figsize=(30, 10))
    plt.subplot(131)
    plt.plot(T_grid, v0[:, Cnum, Fnum])
    plt.xlabel("Temperature (K)")
    plt.title(
        "V, (C, G)=({:.2f},{:.2f})".format(C_grid[Cnum], F_grid[Fnum]))
    plt.subplot(132)
    plt.plot(T_grid, Ca[:, Cnum, Fnum])
    plt.xlabel("Temperature (K)")
    plt.title(
        "g")
    plt.subplot(133)
    plt.plot(T_grid, dvdC[:, Cnum, Fnum]+dvdF[:, Cnum, Fnum])
    plt.xlabel("Temperature (K)")
    plt.title(
        r"$\partial V/\partial C$+$\partial V/\partial F$")
    plt.savefig("test.pdf")


interactive(children=(IntSlider(value=50, description='Fnum'), IntSlider(value=10, description='Cnum', max=20)…